In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from scapy.all import *
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab

In [ ]:
Loss_Network = [0.1, 1, 2.5, 5, 10, 20]

In [ ]:
def ReadCapture(capture_name):
    Relevant_Packets = []
    with PcapReader(capture_name) as pcap_reader:
        for pkt in pcap_reader:
            if IP in pkt:
                ip_src = pkt[IP].src
                if (ip_src == "10.10.0.2"):
                    if pkt.haslayer(UDP):
                        Relevant_Packets.append(pkt)
    return Relevant_Packets

In [ ]:
def GetAllBandwidths(Relevant_Packets, loss, isClient):
    if (isClient == True):
        bandwidth_filename = "bandwidth_" + str(loss) + "_client.txt"
    else:
        bandwidth_filename = "bandwidth_" + str(loss) + "_server.txt"        

    for i in range(0, len(Relevant_Packets)):
        Bandwidth_Packets = []
        pkt_1 = Relevant_Packets[i]
        time_1 = pkt_1.time
        Bandwidth_Packets.append(pkt_1)
        for j in range(i+1, len(Relevant_Packets)):
            pkt_2 = Relevant_Packets[j]
            time_2 = pkt_2.time
            if (time_2 - time_1 > 1.0):
                break
            else:
                Bandwidth_Packets.append(pkt_2)
        if (len(Bandwidth_Packets) > 1):
            bandwidth = CalculateBandwidth(Bandwidth_Packets, bandwidth_filename)
        else:
            print("No bandwidth was recorded at time " + str(time_1))
    return

In [ ]:
def CalculateBandwidth(Bandwidth_Packets, bandwidth_filename):
    time_1 = Bandwidth_Packets[0].time
    time_2 = Bandwidth_Packets[len(Bandwidth_Packets) - 1].time
    packet_sizes = 0
    #Add up the sizes of all packets occuring in a range of at most 1 second after the first one
    for packet in Bandwidth_Packets:
        packet_sizes += packet[UDP].len
    bandwidth = (packet_sizes/(time_2 - time_1)) #in bytes/sec
    with open(bandwidth_filename, "a") as file:
        file.write(str(bandwidth) + "\n")
    return bandwidth

In [ ]:
def cdf_bandwidth(data_bandwidth):#, max_delay):
    ecdf = sm.distributions.ECDF(data_bandwidth)
    bandwidth = np.linspace(min(data_bandwidth), 3500)#max_delay)
    y_bandwidth = ecdf(bandwidth)
    plt.plot(bandwidth, y_bandwidth, color='b', label='Bandwidth')
    plt.grid(True)
    plt.title('Cumulative Distribution Function')
    pylab.legend(loc='lower right')
    plt.ylabel("Cumulative Fraction of Packets")
    plt.xlabel("Bandwidth Value")
    plt.show()
    return bandwidth, y_bandwidth

def Get95thPercentile(bandwidth, y_bandwidth):
    for i in range(0, len(y_bandwidth)):
        if (y_bandwidth[i] >= 0.95):
            return bandwidth[i]

In [ ]:
def main():
    for loss in Loss_Network:
        """capture_name_client = "LimitedRetransmissions/Loss=" + str(loss) + "/captured-packets_client.pcap"
        capture_name_server = "LimitedRetransmissions/Loss=" + str(loss) + "/captured-packets_server.pcap"
        Relevant_Packets_client = ReadCapture(capture_name_client)
        GetAllBandwidths(Relevant_Packets_client, loss, True)
        Relevant_Packets_server = ReadCapture(capture_name_server)
        GetAllBandwidths(Relevant_Packets_server, loss, False)"""
        
        print("Loss rate:" + str(loss) + "%")
        
        bandwidth_client_txt = "Bandwidths/bandwidth_" + str(loss) + "_client.txt"
        data_bandwidth_loss_client = np.loadtxt(bandwidth_client_txt)
        print("Client")
        print("Mean: " + str(np.mean(data_bandwidth_loss_client)))
        print("Median: " + str(np.median(data_bandwidth_loss_client)))
        (bandwidth_client, y_bandwidth_client) = cdf_bandwidth(data_bandwidth_loss_client)
        bandwidth_95_client = Get95thPercentile(bandwidth_client, y_bandwidth_client) 
        print("95% percentile: " + str(bandwidth_95_client) + "\n\n")

        bandwidth_server_txt = "Bandwidths/bandwidth_" + str(loss) + "_server.txt"
        data_bandwidth_loss_server = np.loadtxt(bandwidth_server_txt)
        print("Server")
        print("Mean: " + str(np.mean(data_bandwidth_loss_server)))
        print("Median: " + str(np.median(data_bandwidth_loss_server)))
        (bandwidth_server, y_bandwidth_server) = cdf_bandwidth(data_bandwidth_loss_server)
        bandwidth_95_server = Get95thPercentile(bandwidth_server, y_bandwidth_server) 
        print("95% percentile: " + str(bandwidth_95_server) + "\n\n")
    return

In [ ]:
if __name__ == "__main__":
    main()